**Определение автора с самой высокой оценкой книг (из книг с 50 и более оценками)**

In [ ]:
# формирование запроса в SQL
query = '''        
SELECT a.author AS author,
       c.title AS title,
       p.count AS reviews,
       p.avg AS rating
FROM
    (SELECT book_id,   -- выберем id книги
     COUNT(rating),    -- посчитаем количество оценок
     AVG(rating)       -- и средний рейтинг книги
     FROM ratings      -- из таблицы рейтингов
     GROUP BY book_id            -- сгруппируем по id книги
     HAVING COUNT(rating) > 50   -- оставим только те книги, у которых больше 50 оценок
     ORDER BY AVG(rating) DESC   -- отсортируем по убыванию рейтинга
     LIMIT 1) AS p               -- оставляем только одну лучшу книгу
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT book_id,            -- id книги и
            title,
            author_id           -- id автора
    FROM books) AS c ON p.book_id = c.book_id
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT *
     FROM authors) AS a ON c.author_id = a.author_id ;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine

**Подсчет среднего количества обзоров от пользователей, поставивших более 50 оценок**

In [ ]:
# формирование запроса в SQL
query = '''        
SELECT AVG(c.reviews_count)       -- считаем среднее количество обзоров
FROM
    (SELECT username,            -- выбираем имена пользователей 
     COUNT(rating)  AS ratings_count    -- считаем сколько они раз ставили оценки
     FROM ratings                       -- из таблицы оценок
     GROUP BY username                  -- группируем по имени пользователя 
     HAVING COUNT(rating) > 50) AS p    -- оставляем только тех, кто оставил больше 50 оценок
LEFT OUTER JOIN                 -- к получившейся информации присоединяем 
    (SELECT username,           -- информацию о пользователях
       COUNT (DISTINCT text) AS reviews_count    -- и том, сколько отзывов они оставили
FROM reviews
GROUP BY username) AS c ON p.username = c.username ;
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)

**Поиск интервала между регистрацией и временем создания первого поста.**

In [ ]:
# формирование запроса в SQL
query = '''        
WITH rd AS (                             -- создадим временную таблицу
    SELECT id, creation_date             -- выберем id и дату регистрации пользователя
    FROM stackoverflow.users),
    
posts AS (
    SELECT user_id, creation_date,       -- выберем id пользователя и дату создания поста
    ROW_NUMBER () OVER (PARTITION BY user_id ORDER BY creation_date) AS post_number  -- пронумеруем посты пользователя
    FROM stackoverflow.posts)
    
SELECT user_id,                           -- выберем id пользователя
p.creation_date - r.creation_date         -- разницу между датой поста и датой регистрации 
FROM posts AS p JOIN rd AS r ON p.user_id = r.id     -- присоединив к постам дату регистрации
WHERE post_number = 1;                               -- выбрав разницу только для первого поста
'''

# выполнение запроса в pandas 
pd.io.sql.read_sql(query, con = engine)